In [98]:
#Importing relevant modules
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import numpy as np
from joblib import load, dump
import shap
import matplotlib.pyplot as plt
from PIL import Image, ImageTk

In [99]:
# Setting matplotlib to use 'Agg' backend
#Non-interactive backend
plt.switch_backend('Agg')


Loading the random forest models trained to predict the age of leukaemia onset from the age of onset of other broad disease categories (first occurrence chapters) within the UK Biobank.

In [100]:
# Loading the trained models for IMD high and low categories
loaded_rf_final_high = load("C:/Users/Aakanksha Choudhary/OneDrive/Desktop/project/rf_final_high_model.joblib")
loaded_rf_final_low = load("C:/Users/Aakanksha Choudhary/OneDrive/Desktop/project/rf_final_low_model.joblib")

# Defining the features that the user will input
features = ['infectious_parasitic', 'blood', 'endocrine_system', 'mental', 'nervous_system', 'eye', 'ear',
            'circulatory_system', 'respiratory_system', 'digestive_system', 'skin', 'musculoskeletal_system', 'genitourinary_system', 'pregnancy',
            'perinatal', 'congenital']

Creating a function to display the SHAP force plot for each prediction. The force plot for each individual prediction shows the direction that each contributing feature pushes the prediction in (negative or positive) 

In [101]:
def display_shap_plot(explainer, shap_values, input_data):
    # Creating a force plot
    shap.force_plot(explainer.expected_value, shap_values, input_data, matplotlib=True)
    plt.savefig("shap_force_plot.png", bbox_inches='tight', dpi=70)  # Saving the plot as an image
    plt.close()

    # Displaying the SHAP plot in a new Tkinter window
    shap_window = tk.Toplevel(root)
    shap_window.title("SHAP Force Plot")

    img = Image.open("shap_force_plot.png")
    img = ImageTk.PhotoImage(img)

    panel = tk.Label(shap_window, image=img)
    panel.image = img  # Keeiping a reference to avoid garbage collection
    panel.pack()

Creating a function to predict the age of leukaemia onset by utilising the random forest models. The user can input the ages, choose the category based on the deprivation (high and low) and a message box will pop up desplaying the predicted age.



In [102]:
def predict_age_of_cancer_onset():
    try:
        # Empty list to store the user input values
        input_data = []
        for feature in features:
            # Getting the user value for the current feature
            value = entries[feature].get()
            # If no input value, assuming the age to be 999
            if value == '':
                input_data.append(999)
            else:
                input_data.append(float(value))

        # Creating DataFrame for prediction
        input_df = pd.DataFrame([input_data], columns=features)

        # Selecting model based on user choice
        if model_choice.get() == "High":
            model = loaded_rf_final_high
        else:
            model = loaded_rf_final_low

        # Prediction
        prediction = model.predict(input_df)[0]

        # SHAP explanation
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(input_df)

        # Displaying the prediction
        messagebox.showinfo("Prediction", f"Predicted Age of Cancer Onset: {prediction:.2f} years")

        # Display SHAP plot
        display_shap_plot(explainer, shap_values, input_df.iloc[0, :])

    except ValueError as ve:
        messagebox.showerror("Value Error", str(ve))
    except Exception as e:
        messagebox.showerror("Error", str(e))


GUI CREATION USING TKINTER

In [103]:
# Creating the main window
root = tk.Tk()
root.title("Age of Cancer Onset Prediction")
root.configure(bg='lightblue')
root.geometry("500x600")

# Defining font style and size for labels
label_font = ("Arial", 10)


In [104]:
# Creating input fields for each feature
#Empty dictionary to store user entries
entries = {}
for feature in features:
    #Root frame
    frame = tk.Frame(root)
    frame.pack(padx=10, pady=10)

    #Label formatting
    label = tk.Label(frame, text=feature, width=20, anchor='w')
    label.pack(side='left', padx=5)

    #Creating the entry widget
    entry = tk.Entry(frame)
    entry.pack(side='right', padx=5)

    #Assigning entry values to each feature and storing the feature as a key and the user entry as a value in the dictionary
    entries[feature] = entry



In [105]:
# Creating a frame for model selection
model_frame = tk.Frame(root, bg='lightblue')
model_frame.pack(padx=10, pady=10)

# Creating radio buttons for model selection
model_choice = tk.StringVar(value="High")
high_model_rb = tk.Radiobutton(model_frame, text="High Model", variable=model_choice, value="High", bg='lightblue', font=label_font)
low_model_rb = tk.Radiobutton(model_frame, text="Low Model", variable=model_choice, value="Low", bg='lightblue', font=label_font)
high_model_rb.pack(side='left', padx=5)
low_model_rb.pack(side='right', padx=5)

In [ ]:
# Creating the prediction button
#Takes in the predict age of cancer onset fu
predict_button = tk.Button(root, text="Predict", command=predict_age_of_cancer_onset, bg='lightblue', font=label_font)
predict_button.pack(pady=20)

# Start the Tkinter event loop
root.mainloop()